In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, sum
from pyspark.ml.feature import StringIndexer
from pyspark.sql import DataFrame
from pyspark.sql.types import IntegerType

In [ ]:
spark = SparkSession.builder \
        .appName("KafkaSparkStreaming") \
        .master("spark://spark-master:7077") \
        .getOrCreate()

24/11/07 12:25:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
# df = spark.read.parquet("hdfs://namenode:9000/raw_data")

In [ ]:
df = spark.read.parquet("hdfs://localhost:9000/raw_data")

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
df = df.withColumn('is_injured',
                   when((col('number_of_persons_injured').cast(IntegerType()) +
                         col('number_of_persons_killed').cast(IntegerType())) > 0, 1).otherwise(0))

In [ ]:
# Xoa nhung cot khong can thiet
cols_to_drop = ['latitude', 'longitude', 'on_street_name', 'off_street_name', 'number_of_persons_injured', 'number_of_pedestrians_killed',
                'number_of_cyclist_injured', 'number_of_motorist_injured', 'number_of_motorist_killed',
                'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5',
                'crash_date', 'crash_time', 'number_of_persons_killed', 'number_of_pedestrians_injured',
                'number_of_cyclist_killed']
df_dropped_cols = df.drop(*cols_to_drop)

In [ ]:
#Kiem tra du lieu thieu
missing_counts = df_dropped_cols.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
missing_counts.show()

In [ ]:
#Fill du lieu thieu bang gia tri mode
categorical_cols = ['contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
                    'vehicle_type_code1', 'vehicle_type_code2', 'borough']

for col_name in categorical_cols:
    mode_value = df_dropped_cols.groupBy(col_name).count().orderBy('count', ascending=False).first()[0]
    df_dropped_cols = df_dropped_cols.fillna({col_name: mode_value})

In [ ]:
#Ma hoa du lieu bien phan loai
def encode_categorical_cols(df: DataFrame, categorical_cols: list) -> DataFrame:
    for col_name in categorical_cols:
        indexer = StringIndexer(inputCol=col_name, outputCol=col_name + "_encoded", handleInvalid='keep')
        df = indexer.fit(df).transform(df)
    return df

df_encoded = encode_categorical_cols(df_dropped_cols, categorical_cols)

In [ ]:
df_final = df_dropped_cols.drop(*categorical_cols)

In [ ]:
# feature
x = df_final.drop("is_injured")
# label
y = df_final.select("is_injured")